<a href="https://colab.research.google.com/github/AbhiMalappa/NLP/blob/main/helpdesk_BERT_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from torch import nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch

! pip install -q streamlit
import streamlit as st

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 113.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.6 MB/s eta 0:00:00


In [2]:
! pip install -q streamlit
import streamlit as st

In [25]:
# funct to get predictions
model_ckpt = "drive/MyDrive/Colab Notebooks/NLP/trained_model/Finetune_BERT_for_helpdesk_issue_classification"
#model_ckpt = "drive/MyDrive/Colab Notebooks/NLP/trained_model/V2_Finetune_BERT_for_helpdesk_issue_classification_7_class"

# prediction for all labels
def get_pred(text):
  device = torch.device("cuda")

  #read model
  model = AutoModelForSequenceClassification.from_pretrained(model_ckpt).to(device)

  # from model config get labels names
  labels = model.config.id2label.values()
  labels = list(labels)

  #tokenize input
  tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
  input_encoded = tokenizer(text, return_tensors='pt').to(device)

  # get model pred on tokenized input
  with torch.no_grad():
    outputs = model(**input_encoded) # output is a torch tensor with logit values on gpu

  #output logits
  logits = outputs.logits[0] # get logit values
  #output probs
  probabilities = nn.functional.softmax(logits, dim=-1) # convert logit to prob

  # return a dic with labels and probs
  probabilities = probabilities.cpu().detach().numpy() # convert to numpy on cpu
  result = {labels[i]: probabilities[i] for i in range(len(labels))}

  return result, #probabilities

# prediction return top label
def pipeline_clf(text):
  device = torch.device("cuda")
  clf = pipeline('text-classification',model = model_ckpt)
  pipeline_clf_label = clf(text)[0]['label']
  pipeline_clf_score = clf(text)[0]['score']
  return pipeline_clf_label, pipeline_clf_score

# get sentiment
def sentiment(text):
  sentement_obj = pipeline('text-classification', model = "cardiffnlp/twitter-roberta-base-sentiment-latest")
  #sentement_obj = pipeline('sentiment-analysis')
  #emotions
  #sentement_obj = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=5)

  sentiment = sentement_obj(text)
  return sentiment

# pass custom input
def custom_input(text):
  pred_confidence = {}
  sentiment_val = sentiment(text)
  pred = get_pred(text)[0]
  classification_list = ['agent_unprofessional','long_holdtime','multiple_contacts_made','no_resolution','service_portal', 'Others']

  for keys in pred:
    if keys in classification_list and pred[keys] > 0.2:
      pred_confidence[keys] = pred[keys]

  if len(pred_confidence) == 0:
    pred_confidence['Others'] = 0.3

  if sentiment_val[0]['score'] > 0.6:
    pred_confidence[sentiment_val[0]['label'] ] = sentiment_val[0]['score']

  return pred_confidence, pred


### Predict on new data

Add code to split int scentence and then predict

In [51]:
import pandas as pd
import numpy as np
df = pd.read_csv('drive/MyDrive/Colab Notebooks/training_text_calssifiaction.csv')
df = df.sample(3)

# call prediction
df['pred_label'] = df['string_value'].apply(lambda x: pipeline_clf(x)[0])
df['pred_label_score'] = df['string_value'].apply(lambda x: pipeline_clf(x)[1])
df['pred_label'] = np.where(df['pred_label_score'] < 0.6, 'Others', df['pred_label'])

df['agent_unprofessional_score'] = df['string_value'].apply(lambda x: get_pred(x)[0]['agent_unprofessional'])
df['long_holdtime_score'] = df['string_value'].apply(lambda x: get_pred(x)[0]['long_holdtime'])
df['multiple_contacts_made_score'] = df['string_value'].apply(lambda x: get_pred(x)[0]['multiple_contacts_made'])
df['no_resolution_score'] = df['string_value'].apply(lambda x: get_pred(x)[0]['no_resolution'])
df['service_portal_score'] = df['string_value'].apply(lambda x: get_pred(x)[0]['service_portal'])
df['pred_val_all_class'] = df['string_value'].apply(lambda x: get_pred(x))

df['sentiment_label'] = df['string_value'].apply(lambda x: sentiment(x)[0]['label'])
df['sentiment_score'] = df['string_value'].apply(lambda x: sentiment(x)[0]['score'])

df.head(2)

Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when in

,string_value,Category,pred_label,pred_label_score,agent_unprofessional_score,long_holdtime_score,multiple_contacts_made_score,no_resolution_score,service_portal_score,pred_val_all_class,sentiment_label,sentiment_score
240,He was very patient with me,Others,positive_feedback,0.691563,0.119301,0.021455,0.041322,0.023713,0.014018,"({'agent_unprofessional': 0.11930073, 'long_ho...",positive,0.926954
4768,"Issue not resolved, but the ticket was closed.",no_resolution,no_resolution,0.958546,0.003521,0.003498,0.008349,0.958546,0.002351,"({'agent_unprofessional': 0.003520502, 'long_h...",negative,0.649703


### Predict on text input

In [26]:
text = [
    " I felt like the service rep was not patient enough to explain what the issue was and how i can prevent it from happening again. Probably having a bad day. I was not satisfied with this service. "
    ]

text = [
    "  i want to sleep  "
    ]

custom_input(text)[0]


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


{'long_holdtime': 0.21355872, 'neutral': 0.7814059257507324}

In [16]:
get_pred(text)[0]

{'Others': 0.026007662,
 'agent_unprofessional': 0.040137194,
 'long_holdtime': 0.9233131,
 'no_resolution': 0.0067846593,
 'service_portal': 0.0037572933}

### Zero-Shot Classification

In [30]:
pipe = pipeline(model="facebook/bart-large-mnli")
pipe("I had to talk to 3 different IT Support technicians. Got several different view points. The last technician was about to make a few changes that speed up my results. I was happy with that because it took 1 1/2 days to resolve my issue. I was very happy on the last attempt to resolve my problem.",
    candidate_labels=["unprofessional", "wait", "not resolved", "positive", "negative", "multiple contacts", "lack knowledge"],
)

Device set to use cuda:0


{'sequence': 'I had to talk to 3 different IT Support technicians. Got several different view points. The last technician was about to make a few changes that speed up my results. I was happy with that because it took 1 1/2 days to resolve my issue. I was very happy on the last attempt to resolve my problem.',
 'labels': ['multiple contacts',
  'positive',
  'wait',
  'lack knowledge',
  'negative',
  'unprofessional',
  'not resolved'],
 'scores': [0.7935426235198975,
  0.15460607409477234,
  0.020611831918358803,
  0.015902787446975708,
  0.007993784733116627,
  0.003712371224537492,
  0.0036305116955190897]}

In [ ]:
%%writefile app.py
import streamlit as st

x = st.slider('Select a value')
st.write(x, 'squared is', x * x)

Writing app.py


In [ ]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
added 22 packages in 2s
⠹
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸

In [ ]:
!wget -q -O - ipv4.icanhazip.com
#!curl https://loca.lt/mytunnelpassword


34.48.16.24


In [ ]:
#!streamlit run /content/app.py &>/content/logs.txt &
!streamlit run app.py &>/content/logs.txt &


In [ ]:
!npx localtunnel --port 8501

⠙your url is: https://cute-beers-say.loca.lt
y
34.48.16.24
^C
